In [177]:
import pandas as pd 
import numpy as np
import yfinance as yf 
from datetime import datetime, timedelta

pd.set_option("display.max_columns", 41)

## Getting Price data 

In [178]:
# List of securities 
securities = [ 
    # Equities 
    'SPY', 
    'QQQ',
    'IWM',
    'EZU',
    'FEZ',
    'EWG',
    'EWU',
    'EWJ',
    'MCHI',
    'TSLA',

    # Bonds 
    'TLT',
    'IEF',
    'SHY',
    'LQD',
    'HYG',
    'BNDX',
    'TIP',
    'EMB',
    

    # Commodities    
    "USO",
    "UNG",
    "GLD",
    "CPER",
    "SLV",
    "DBA",
    "WEAT",
    "CORN",
    'SOYB',

    # FX 
    'UUP',
    'FXE',
    'FXY',
    'FXB',
    'FXF',
    'FXC',
    'FXA',
    
    # Crypto 
    'GBTC'
    ]

In [179]:
# Setting the start and end date 
endDate = datetime.today()
startDate = endDate - timedelta(days=50*365)
print(startDate)

1974-07-29 11:20:14.556839


In [180]:
# Downloading stock data to df
close_df = pd.DataFrame()
for i in securities:
    data = yf.download(i, start=startDate, end=endDate)
    close_df[i] = data["Adj Close"]


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [181]:
# Function to calculate 1 month ema's 
def ema4_16(column,df):
    emadf= pd.DataFrame()

    emadf[f"{column} 4ema"]=df.ewm(span=4, adjust=False).mean()
    emadf[f"{column} 16ema"]=df.ewm(span=16, adjust=False).mean()
    
    return emadf

In [182]:
# Function to calculate 3 month ema's 
def ema16_64(column,df):
    emadf= pd.DataFrame()

    emadf[f"{column} 16ema"]=df.ewm(span=16, adjust=False).mean()
    emadf[f"{column} 64ema"]=df.ewm(span=64, adjust=False).mean()
    
    return emadf

In [183]:
# Function to calculate 12 month ema's 
def ema64_256(column,df):
    emadf= pd.DataFrame()

    emadf[f"{column} 64ema"]=df.ewm(span=64, adjust=False).mean()
    emadf[f"{column} 256ema"]=df.ewm(span=256, adjust=False).mean()
    
    return emadf

In [184]:
# Creating 1 month df 
monthly_ema_list = []
for i in close_df:
    emadf=ema4_16(i,close_df[i])
    monthly_ema_list.append(emadf)

In [185]:
# Creating 3 month df 
month3_ema_list = []
for i in close_df:
    emadf_3=ema16_64(i,close_df[i])
    month3_ema_list.append(emadf_3)

In [186]:
# Creating annual df 
annual_ema_list = []
for i in close_df:
    annual_ema=ema64_256(i,close_df[i])
    annual_ema_list.append(annual_ema)

In [187]:
# Calculates if there's uptrend/downtrend 
def trend(df):
    trend = []
    for i in df:
        trend_value = i[f"{i.columns[0]}"] - i[f"{i.columns[1]}"]
        trend.append(trend_value)
    return trend 

In [188]:
# Calculates the volatility in price terms  
def standard_dev_adjustment(df):

    daily_pct_returns = close_df.pct_change().dropna()

    for i in daily_pct_returns:
        price_vol = daily_pct_returns.rolling(window=2).std()
        for j in df:
            dev_adjust=price_vol*df

    return dev_adjust

In [189]:
# Volatility of ema 
def vol_adjusted_ema(trend, stdev):
    return trend / stdev

In [190]:
# Returns scaled forecast 
def forecast(scalar, vol_ema):
    return scalar * vol_ema

In [191]:
# Capping forecast values
# Stops portfolio from having unpropotionally large position 
def capped_forecast(forecast):
    return np.clip(forecast,-20,20)  

In [192]:
# Adding column names of each ticker to a list  
column_names = []
for i in monthly_ema_list:
    column_names.append(i.columns[0][:-5])

In [193]:
# Converting the list of trend values for each ticker into a df 
def convert_list_df(df_trend, columns):
    trend_ema_df = pd.DataFrame()
    counter = 0
    for i in columns:
        trend_ema_df[f'{i}'] = df_trend[counter]
        counter += 1 
    return trend_ema_df

In [194]:
# 1 month data processing   
monthly_trend_ema = trend(monthly_ema_list)
monthly_trend_ema_df = convert_list_df(monthly_trend_ema,column_names)
stdev_adj = standard_dev_adjustment(close_df)
monthly_vol_ema = vol_adjusted_ema(monthly_trend_ema_df,stdev_adj)

In [195]:
# 3 month data processing   
month3_trend_ema = trend(month3_ema_list)
month3_trend_ema_df = convert_list_df(month3_trend_ema,column_names)
month3_vol_ema = vol_adjusted_ema(month3_trend_ema_df,stdev_adj)

In [196]:
# Annual data processing   
annual_trend_ema = trend(annual_ema_list)
annual_trend_ema_df = convert_list_df(annual_trend_ema,column_names)
annual_vol_ema = vol_adjusted_ema(annual_trend_ema_df,stdev_adj)

In [197]:
# Scalar values 
month_1_scalar = 7.5
month_3_scalar = 3.75
annual_scalar = 1.87

In [198]:
# Capped & scaled forecast 
forecast_1_month = forecast(month_1_scalar, monthly_vol_ema)
forecast_1_month= capped_forecast(forecast_1_month)
forecast_1_month

,SPY,QQQ,IWM,EZU,FEZ,EWG,EWU,EWJ,MCHI,TSLA,TLT,IEF,SHY,LQD,HYG,BNDX,TIP,EMB,USO,UNG,GLD,CPER,SLV,DBA,WEAT,CORN,SOYB,UUP,FXE,FXY,FXB,FXF,FXC,FXA,GBTC
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09,20.000000,20.000000,0.207396,5.497594,4.246951,7.948926,4.256740,16.514470,-2.761771,20.000000,-0.119343,18.334557,20.000000,20.000000,20.000000,9.766731,20.000000,12.789577,20.000000,-20.000000,5.637469,20.000000,18.392004,-2.653847,-7.302335,-10.984218,-14.813441,-9.685440,20.00000,-20.000000,20.000000,-4.530725,20.000000,20.000000,-20.000000
2024-07-10,17.227907,20.000000,1.981033,1.872509,1.844403,2.899396,2.403308,16.938206,-3.212364,20.000000,1.454407,14.401325,20.000000,10.093719,12.011873,6.023597,9.426617,6.106116,7.035464,-20.000000,20.000000,15.868301,20.000000,-2.396856,-12.075515,-20.000000,-20.000000,-2.385647,20.00000,-20.000000,10.662448,-7.467146,20.000000,20.000000,-16.770479
2024-07-11,7.265871,5.701027,5.457281,6.554876,5.576221,17.582096,20.000000,10.102126,0.498892,12.413579,6.900273,9.632949,14.302188,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,-20.000000,9.461300,4.769221,14.667678,-0.348387,-5.265317,-20.000000,-13.414992,-5.400685,20.00000,-0.870801,20.000000,-0.187604,18.080574,20.000000,-20.000000


In [199]:
# Capped & scaled forecast 
forecast_3_month = forecast(month_3_scalar, month3_vol_ema)
forecast_3_month= capped_forecast(forecast_3_month)
forecast_3_month

,SPY,QQQ,IWM,EZU,FEZ,EWG,EWU,EWJ,MCHI,TSLA,TLT,IEF,SHY,LQD,HYG,BNDX,TIP,EMB,USO,UNG,GLD,CPER,SLV,DBA,WEAT,CORN,SOYB,UUP,FXE,FXY,FXB,FXF,FXC,FXA,GBTC
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09,20.000000,20.000000,0.256976,1.570338,1.742041,4.363951,16.665280,3.977182,0.919839,16.686583,6.571374,20.000000,20.000000,20.000000,20.000000,11.769702,20.000000,14.278667,20.000000,-4.675921,5.040600,20.000000,16.889230,-0.801498,-7.903681,-7.635416,-16.262961,20.000000,7.782363,-20.000000,20.000000,4.022552,8.730968,14.68862,-6.815406
2024-07-10,19.784858,20.000000,0.487969,0.455550,0.607301,1.397071,4.257561,4.446707,0.991642,17.491421,5.760016,17.376422,20.000000,10.170061,12.925527,5.117701,12.492743,5.610513,8.170230,-20.000000,20.000000,5.974709,20.000000,-0.809643,-11.140586,-20.000000,-20.000000,17.505265,4.232261,-20.000000,6.927045,2.829355,20.000000,20.00000,-6.174065
2024-07-11,9.665267,8.096335,0.990839,1.650599,1.809025,7.418274,20.000000,3.156226,0.935662,7.460204,7.266498,8.133642,14.183159,15.917355,20.000000,20.000000,20.000000,20.000000,20.000000,-20.000000,5.937436,2.967786,12.715466,-0.546767,-5.928718,-19.941958,-11.017247,9.241548,7.396307,-5.303565,20.000000,0.793578,7.423546,20.00000,-20.000000


In [200]:
# Capped & scaled forecast 
annual_forecast = forecast(annual_scalar, annual_vol_ema)
annual_forecast= capped_forecast(annual_forecast)
annual_forecast

,SPY,QQQ,IWM,EZU,FEZ,EWG,EWU,EWJ,MCHI,TSLA,TLT,IEF,SHY,LQD,HYG,BNDX,TIP,EMB,USO,UNG,GLD,CPER,SLV,DBA,WEAT,CORN,SOYB,UUP,FXE,FXY,FXB,FXF,FXC,FXA,GBTC
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09,20.000000,20.000000,9.263018,20.000000,20.000000,20.000000,20.000000,12.331735,1.119989,-3.726506,-5.183397,4.061910,20.00000,20.000000,20.0,20.000000,20.000000,20.000000,20.000000,-20.000000,13.780875,20.000000,20.000000,8.700675,-5.046566,-6.20678,-12.890235,20.000000,20.000000,-20.000000,20.000000,-8.278931,1.949133,5.764624,20.000000
2024-07-10,20.000000,20.000000,7.363509,6.180869,6.624015,5.948916,8.998017,9.886255,1.432288,-2.960803,-4.366620,3.250803,20.00000,10.447207,20.0,12.339272,7.959227,10.707984,6.393032,-20.000000,20.000000,20.000000,20.000000,7.126920,-6.900479,-20.00000,-20.000000,20.000000,8.474564,-20.000000,8.106046,-7.627955,20.000000,20.000000,20.000000
2024-07-11,11.820462,8.162031,3.910665,14.748286,15.020428,20.000000,20.000000,5.836596,1.015171,-1.089017,-4.739023,1.588819,16.48218,15.275923,20.0,20.000000,20.000000,20.000000,20.000000,-20.000000,13.593053,12.706678,15.473086,6.199403,-3.715965,-15.53747,-7.980215,17.311228,10.746856,-6.771774,20.000000,-1.788192,1.656721,20.000000,20.000000
